In [11]:
import numpy as np
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [12]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'congress'
))

In [20]:
myquery = '''
SELECT directordername, bioguideid
FROM members
ORDER BY lastname
'''

members = pd.read_sql_query(myquery, engine)
memberlist = [{'label': x, 'value': y} for x, y in zip(members['directordername'], members['bioguideid'])]

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [7]:
mymarkdown = '''

Congressional elections are usually all about the big national issues, but these Representatives and Senators work for local districts and states. Not all issues are the same everywhere. The purpose of this dashboard is to collect public data from these sources:

* [Official API for the U.S. Congress](https://api.congress.gov/)
* [Voteview](https://voteview.com/)
* [Open Secrets](https://opensecrets.org/open-data/api)
* [ProPublica's Data on Bills](https://www.propublica.org/)

'''

In [21]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        html.H1("Know Your Representatives in Congress"),
        html.H2("Data collected from APIs from Congress.gov, Open Secrets, Voteview.com, and ProPublica"),
        html.H3("DS 6600: Data Engineering 1, UVA Data Science"),

        dcc.Markdown(mymarkdown),

        dcc.Dropdown(options = memberlist,
                    value = 'Alma S. Adams')
    ]
)

if __name__=="__main__":
    app.run_server(mode='external', host = "0.0.0.0", port = 8050, debug=False)